In [29]:
tf.set_random_seed(0)  # 乱数シード

# logistic regression
import numpy as np
import tensorflow as tf

# weight, bias
w = tf.Variable(tf.zeros([2, 1]))
b = tf.Variable(tf.zeros([1]))

# model y = sigmmoid(w^T*x + b)
# input x
x = tf.placeholder(tf.float32, shape=[None, 2])
# teacher
t = tf.placeholder(tf.float32, shape=[None, 1])
# output y
y = tf.nn.sigmoid(tf.matmul(x, w) + b) 

cross_entropy = - tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
# 確認
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

# OR logic
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [1]])

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(200):
    sess.run(train_step, feed_dict={
            x: X,
            t: Y
        })

dlassified = correct_prediction.eval(session=sess, feed_dict={
        x: X,
        t: Y
    })
print(classified)

prob = y.eval(session=sess, feed_dict={
        x: X,
        t: Y
    })

print(prob)

# 学習したパラメータの確認
print('w:', sess.run(w))
print('b:', sess.run(b))

[[ True]
 [ True]
 [ True]
 [ True]]
[[ 0.22355038]
 [ 0.91425949]
 [ 0.91425949]
 [ 0.99747425]]
w: [[ 3.61188436]
 [ 3.61188436]]
b: [-1.24509501]


In [34]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

model = Sequential([
    Dense(input_dim=2, output_dim=1),
    Activation('sigmoid')
])